In [4]:
#Loop all accounts all keys  looking for Metrics
df_counts = pd.DataFrame()
df3 = pd.DataFrame()
out =0
superlist = []
countlist = []
accountlist = []
for query_key, account_id in zip(AllAPIKeysList,AllRPMsList) :
    if (query_key != 'none' ): 
        
        query_str = 'select uniques(metricName) from Metric since \'2020-08-01 00:00:00\' until \'2020-09-01 00:00:00\' limit max'
        #query_key = apiKEY
        #account_id = account

        query = { 'nrql': query_str }
        headers = {'X-Query-Key': query_key}
        retry = 3
        url = (
                'https://insights-api.newrelic.com'
                '/v1/accounts/{}/query?{}'.format(account_id, urllib.parse.urlencode(query))
            )
        success = False
        r_json = {}
        count = 0
        while not success and count < retry:
            r = requests.get(url, headers=headers)
            r_json = r.json()
            success = r.status_code == requests.codes.ok
            count = count + 1

        if not success:
            print_error('Error getting INSIGHT query: {}'.format(r_json))
        
        #print(r_json['results'][0]['members'])
        
        for metric in r_json['results'][0]['members']:
            metric_query = 'select count(*) from Metric where metricName = \''+metric+'\' AND (appName like \'%%\' or appName IS NULL) since \'2020-08-01 00:00:00\' until \'2020-09-01 00:00:00\' FACET monthOf(timestamp),metricName,appName,entity.name,tags.appName,Application limit max'
            #metric_query = 'select count(*) from Transaction'
            
            query2 = { 'nrql': metric_query }
            headers2 = {'X-Query-Key': query_key}
            retry2 = 3
            url2 = (
                    'https://insights-api.newrelic.com'
                    '/v1/accounts/{}/query?{}'.format(account_id, urllib.parse.urlencode(query2))
                )
            
            #print(query2)
            success2 = False
            r_json2 = {}
            count2 = 0
            while not success2 and count2 < retry2:
                r2 = requests.get(url2, headers=headers2)
                r_json2 = r2.json()
                success2 = r2.status_code == requests.codes.ok
                count2 = count2 + 1

            if not success2:
                print_error('Error getting INSIGHT query: {}'.format(r_json2))
        
            if len(r_json2['facets']) != 0:
                for name in r_json2['facets']:
                    #print(name)
                    superlist.append(name['name'])
                    for result in name['results']:
                        countlist.append(result['count'])
                        accountlist.append(account_id)
                    #superlist.append(name['results'])
                    
                #df_counts.append(pd.json_normalize(r_json2['facets'],['results']),ignore_index=True)

            out = out +1
            if (out == 3000000):
                break
                
    




#print(countlist)
#print(df_counts)
df3 = pd.DataFrame(superlist, columns = ['Month', 'MetricName','appName','entity.name','tags.appName','Application']) 
#df3 = pd.DataFrame(superlist,columns = ['Month', 'MetricName','appName','entity.name','tags.appName','Application'])

df3['counts'] = countlist
df3['accountId'] = accountlist

df3

            
            

NameError: name 'print_error' is not defined